In [2]:
import numpy as np
from ml_switching_reg_sim.data_creation import MisclassificationCreator, UberDatasetCreator, UberDatasetCreatorHet
from datetime import timedelta


In [34]:
u = UberDatasetCreatorHet(drivers=1000, time_periods=30, regimes=2)

df, mw, [beta0, beta1], y_sd = u.construct(
    seed=1,
    output_true_beta=True,
    output_sigma=True,
    y_sd = [1,1],
    beta1 = [.1, .2],
    beta0 = 1,
    weight=0
    )

beta_regime_0 = np.array([beta0, beta1[0]])
beta_regime_1 = np.array([beta0, beta1[1]])

In [7]:
N = 1000
rng = np.random.default_rng(seed=1)

# X0 = rng.standard_normal(size=(N, 3))
# X1 = rng.standard_normal(size=(N, 3))
# W0 = np.concatenate([np.ones((N,1)), X0], axis=1) # add intercept
# W1 = np.concatenate([np.ones((N,1)), X1], axis=1)
# R = rng.choice([0,1],size=(N,1))
# beta0 = np.array([[1,.1,.2,.3]]).T
# beta1 = np.array([[.9,.2,.3,.4]]).T

# W = ((1-R)*W0@beta0 + R*W1@beta1)

# epsilon = rng.normal(size=(N,1))

# y = W + epsilon

# mw = MisclassificationCreator(regimes=2, seed=1)
# extent = 0
# pi = np.concatenate([mw.noisify_matrix(extent=extent, index=1), mw.noisify_matrix(extent=extent, index=0)], axis=0).reshape(2,2)

R = df['regime'].values
y = df['y'].values

W0 = np.concatenate([np.ones((u.drivers*u.time_periods,1)), df['drought_0'].values.reshape(-1,1)], axis=1) # add intercept
W1 = np.concatenate([np.ones((u.drivers*u.time_periods,1)), df['drought_1'].values.reshape(-1,1)], axis=1)

RR0 = lambda i: ((1-R[i])*mw[0,0] + R[i]*mw[0,1])
RR1 = lambda i: ((1-R[i])*mw[1,0] + R[i]*mw[1,1])


In [17]:
def Pr(r, r_hat, pi, sigma2, y, W0, W1, beta0, beta1):
    
    regime_0 = np.exp(-(y-W0@beta0)**2/(2*sigma2))
    regime_1 = np.exp(-(y-W1@beta1)**2/(2*sigma2))

    F0 = pi[0,0]*regime_0 + pi[1,0]*regime_1
    F1 = pi[0,1]*regime_0 + pi[1,1]*regime_1
    
    if r==0 and r_hat==0:
        return pi[0,0]*regime_0/F0
    elif r==0 and r_hat==1:
        return pi[1,0]*regime_1/F0
    elif r==1 and r_hat==0:
        return pi[0,1]*regime_0/F1
    elif r==1 and r_hat==1:
        return pi[1,1]*regime_1/F1
    else:
        return 0
    
    
    
    
    

In [9]:
num0 = 0
denom0 = 0

num1 = 0
denom1 = 0

for i in range(u.drivers*u.time_periods):
    num0 += RR0(i)*(W0[i].T * y[i])
    denom0 += RR0(i)*(W0[i][np.newaxis, :].T @ W0[i][np.newaxis, :])
    
    num1 += RR1(i)*(W1[i].T * y[i])
    denom1 += RR1(i)*(W1[i][np.newaxis, :].T @ W1[i][np.newaxis, :])
    
print(np.linalg.inv(denom0) @ num0)
print(np.linalg.inv(denom1) @ num1)

[0.98778266 0.09536444]
[0.98334083 0.1920972 ]


## Now with Matrices

In [12]:
np.diag(R)


array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
R1_mat = np.diag(RR1(range(u.drivers*u.time_periods)).flatten())
R0_mat = np.diag(RR0(range(u.drivers*u.time_periods)).flatten())

In [15]:
np.linalg.inv(W1.T @ R1_mat @ W1) @ (W1.T @ R1_mat @ y)


array([0.98334083, 0.1920972 ])

In [16]:
np.linalg.inv(W0.T @ R0_mat @ W0) @ (W0.T @ R0_mat @ y)

array([0.98778266, 0.09536444])

## Using IRLS



In [ ]:
RR0 = lambda i: ((1-R[i])*mw[0,0] + R[i]*mw[0,1])
RR1 = lambda i: ((1-R[i])*mw[1,0] + R[i]*mw[1,1])